In [ ]:
%pip install libtorrent coloredlogs --quiet

In [ ]:
from google.colab import drive, runtime

drive.mount("/content/drive")

In [ ]:
import os
import gc
import re
import json
import time
import shutil
import logging
import coloredlogs
import libtorrent as lt
from IPython.display import display, HTML, clear_output

In [ ]:
# Set the desired timezone (e.g., "America/New_York")
desired_timezone = "Europe/Istanbul"

# Set the timezone using the environment variable
os.environ["TZ"] = desired_timezone

# Call tzset() to apply the changes
time.tzset()

In [ ]:
def create_directory(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

In [ ]:
def pprint(string: str):
    """
    Custom print function tailored for Jupyter Notebook. Writes a string, adds a newline,
    and ensures immediate display with a delay for pacing log messages.

    Args:
    - string (str): The string to print.
    """
    # Wrap the string in HTML for proper display in Jupyter Notebook
    display(HTML(f"{string}<br>"))
    time.sleep(3)  # Delay for pacing log messages
    # Clear the output to prevent flooding the notebook with too many messages
    clear_output(wait=True)

In [ ]:
def validate_magnet_link(magnet_link, exact=False):
    """
    Validate a magnet link based on a specified pattern.

    Args:
    - magnet_link (str): The magnet link to be validated.
    - exact (bool): If True, match the entire string; otherwise, match partial.

    Returns:
    - bool: True if the magnet link is valid, False otherwise.
    """

    # Define the pattern based on exact match requirement
    pattern_str = (
        r"^magnet:\?xt=urn:[a-z0-9]+:[a-z0-9]{32,40}&dn=.+&tr=.+$"
        if exact
        else r"magnet:\?xt=urn:[a-z0-9]+:[a-z0-9]{32,40}&dn=.+&tr=.+"
    )

    # Compile the regular expression pattern
    pattern = re.compile(pattern_str, re.I | re.M)

    # Return True if the magnet link matches the pattern, otherwise False
    return bool(pattern.match(magnet_link))

In [ ]:
def load_magnet_links(file_name: str, logger: logging.Logger) -> set:
    """
    Load and validate magnet links from a file.

    Args:
    - file_name (str): Name of the file containing magnet links.

    Returns:
    - set: A set of valid magnet links.

    Raises:
    - FileNotFoundError: If the specified file is not found.
    """

    try:
        # Read and strip each line in the file
        with open(file_name) as f:
            lines = [line.strip() for line in f]

        # Filter out invalid magnet links
        magnet_links = [line for line in lines if validate_magnet_link(line)]

        # Display the number of valid magnet links loaded
        logger.info(f"Loaded {len(magnet_links)} magnet links from {file_name}")

        return set(magnet_links)

    except FileNotFoundError:
        # Handle file not found error
        logger.error(f"File not found: {file_name}")
        return set([])

In [ ]:
def get_magnet_links(existing_links: set, logger: logging.Logger) -> set:
    """
    Get unique magnet links from user input and validate them.

    Args:
        existing_links (set): Set containing already existing magnet links.

    Returns:
        set: Updated set of magnet links.
    """
    count = 0
    timeout = 60  # seconds
    start_time = time.time()

    try:
        while True:
            link = input("Enter a magnet link or 'exit' to quit: ")

            if link.lower() in ["exit", "quit", "q"]:
                break

            if validate_magnet_link(link, exact=True):
                if link not in existing_links:
                    existing_links.add(link)
                    print(f"Added magnet link: {link}")
                    count += 1
                else:
                    logger.warning(f"Magnet link already added: {link}")

                start_time = time.time()  # Reset the timer
            else:
                logger.warning(f"Invalid magnet link: {link}")

            # TODO: Timer not works properly
            if time.time() - start_time > timeout:
                print("Timeout: No input received in the last 60 seconds.")
                break

    except KeyboardInterrupt:
        pass

    logger.info(f"Added {count} magnet links")
    return existing_links

In [ ]:
def update_txt_file(file_name: str, magnet_links: set, logger: logging.Logger):
    # Extract folder name and file base name
    folder_name, _ = os.path.split(file_name)

    # Create directory if it doesn't exist
    create_directory(folder_name)

    # Write magnet links to the file
    with open(file_name, "w") as f:
        f.write("\n".join(magnet_links))

    # Print a timestamped message
    logger.info(f"Txt file updated -> {file_name}")

In [ ]:
class Torrent:
    def __init__(
        self,
        ses: lt.session,
        magnet_link: str,
        logger: logging.Logger,
        timeout: int = 28800,
    ):
        self.ses = ses
        self.magnet_link = magnet_link
        self.save_path = "/content/Torrents"
        self.timestamp = time.time()
        self.handle = self.add_torrent(ses)
        self.name = self.handle.status().name
        self.timeout = timeout  # seconds
        self.logger = logger

    def add_torrent(self, ses: lt.session):
        atp = lt.parse_magnet_uri(self.magnet_link)
        atp.save_path = self.save_path
        atp.storage_mode = lt.storage_mode_t.storage_mode_sparse
        return ses.add_torrent(atp)

    def remove_torrent(self, ses: lt.session):
        ses.remove_torrent(self.handle)
        # Wait for the torrent to be removed from the session
        time.sleep(5)
        self.logger.info(f"Torrent successfully removed: {self.name}")

    def remove_stale_torrents(self, ses: lt.session):
        if time.time() - self.timestamp > self.timeout:
            self.remove_torrent(ses)
            return True
        return False

    def get_status(self):
        try:
            return self.handle.status()
        except RuntimeError as e:
            self.logger.error(
                f"{e} - Invalid torrent handle used! - Get status - {self.name}"
            )

    def get_progress(self):
        try:
            return self.handle.status().progress * 100
        except RuntimeError as e:
            self.logger.error(
                f"{e} - Invalid torrent handle used! - Progress rate - {self.name}"
            )

    def get_download_rate(self):
        try:
            return self.handle.status().download_rate // 131072
        except RuntimeError as e:
            self.logger.error(
                f"{e} - Invalid torrent handle used! - Download rate - {self.name}"
            )

    def get_folder_name(self, lower=True):
        name = self.name[:50].split(".")[0]
        name = name.split(" ")[0]
        folder_name = re.sub(r"[^a-zA-Z0-9]", "", name)
        return folder_name.lower() if lower else folder_name

    def get_torrent_name(self):
        return self.handle.status().name

    def remove_the_link_from_the_list(self, magnet_links):
        magnet_links.remove(self.magnet_link)

    def remove_files(self):
        try:
            # Get the beginning of the torrent name to match truncated folder names
            folder_name_start = self.name[:40].lower()

            # Log available folders for debugging
            available_folders = os.listdir(self.save_path)
            self.logger.debug(f"Available folders in save path: {available_folders}")

            source_folder = None
            for folder in available_folders:
                if folder.lower().startswith(folder_name_start):
                    source_folder = os.path.join(self.save_path, folder)
                    break

            if not source_folder:
                self.logger.error(
                    f"Can't find source folder at the save path! Can't remove files. - Torrent name: {self.name}"
                )
                return False

            self.logger.debug(f"Source folder found: {source_folder}")

            shutil.rmtree(source_folder)
            self.logger.info(
                f"Files successfully removed from session: {source_folder}"
            )
            return True

        except Exception as e:
            self.logger.error(f"Session does not contain files. {e}")
            return False

    def is_download_complete(self):
        status = self.get_status()
        return status.state == lt.torrent_status.seeding

    def move_video_files(self, destination: str):
        if not self.is_download_complete():
            self.logger.error(f"Torrent not fully downloaded: {self.name}")
            return False

        torrent_destination = os.path.join(destination, self.get_folder_name())
        os.makedirs(torrent_destination, exist_ok=True)

        # Get the beginning of the torrent name to match truncated folder names
        folder_name_start = self.name[:40].lower()

        # Log available folders for debugging
        available_folders = os.listdir(self.save_path)
        self.logger.debug(f"Available folders in save path: {available_folders}")

        source_folder = None
        for folder in available_folders:
            if folder.lower().startswith(folder_name_start):
                source_folder = os.path.join(self.save_path, folder)
                break

        if not source_folder or not os.path.exists(source_folder):
            self.logger.error(
                f"Can't find source folder at the save path! Can't move files. - Torrent name: {self.name}"
            )
            return False

        self.logger.debug(f"Source folder found: {source_folder}")

        moved_files = 0
        video_extensions = (".mp4", ".mkv", ".avi", ".mov", ".srt")

        for file in os.listdir(source_folder):
            if file.lower().endswith(video_extensions):
                source_path = os.path.join(source_folder, file)
                destination_path = os.path.join(torrent_destination, file)
                shutil.move(source_path, destination_path)
                moved_files += 1
                self.logger.info(f"Moved file: {source_path} to {destination_path}")
                time.sleep(1)

        shutil.rmtree(source_folder, ignore_errors=True)
        time.sleep(1)
        self.logger.info(f"Moved {moved_files} video files to: {torrent_destination}")
        return moved_files > 0

In [ ]:
def create_torrents(
    ses: lt.session, magnet_links: set, downloads: list, logger: logging.Logger
) -> set:
    """
    Create and add new torrents to the session.

    Args:
    - ses (lt.session): libtorrent session
    - magnet_links (set): Set of magnet links
    - downloads (list): List of existing torrent objects

    Returns:
    - set: Updated set of magnet links
    """

    logger.info("Starting to create and add torrents to the session.")

    created_count = 0

    for magnet_link in magnet_links:
        if magnet_link in [torrent.magnet_link for torrent in downloads]:
            logger.warning(f"Torrent already added to session: {magnet_link[:120]}")

        else:
            downloads.append(Torrent(ses, magnet_link, logger))
            created_count += 1
            # Increase sleep time for stability
            time.sleep(5)

    logger.info(f"Successfully created {created_count} new torrents.")

In [ ]:
def check_already_downladed(
    ses: lt.session,
    downloads: list,
    destination: str,
    magnet_links: set,
    magnet_links_txt: str,
    logger: logging.Logger,
):
    logger.info("Checking if a torrent already downloaded..")

    for torrent in downloads:
        folder_path = os.path.join(destination, torrent.get_folder_name())

        if os.path.exists(folder_path):
            downloadeds = os.listdir(folder_path)

            if any(torrent.name[:50] in downloaded[:50] for downloaded in downloadeds):
                torrent.remove_torrent(ses)
                torrent.remove_files()
                torrent.remove_the_link_from_the_list(magnet_links)
                update_txt_file(magnet_links_txt, magnet_links, logger)
                torrent_name = torrent.name
                downloads.remove(torrent)
                logger.warning(
                    f"Torrent already downloaded: {torrent_name} and removed from session!!!\nIt's already in {folder_path}."
                )
                gc.collect()

In [ ]:
def display_torrent_progress(torrent: Torrent):
    """
    Displays the progress, download speed, and name for each active torrent using the custom pprint function.

    Args:
    - torrent (Torrent): A Torrent objects currently being downloaded.
    """
    # Torrent name
    name = torrent.get_torrent_name()

    # Calculate download percentage
    progress = torrent.get_progress()

    # Download speed in MB/s
    download_speed = torrent.get_download_rate()

    # Use pprint to display the information on a new line each time
    pprint(
        f"{time.strftime('%H:%M:%S')} - {name}: {progress:.2f}% complete, Download speed: {download_speed:.2f} KB/s"
    )

In [ ]:
def download_torrents(
    ses: lt.session,
    downloads: list,
    destination: str,
    magnet_links: set,
    magnet_links_txt: str,
    logger: logging.Logger,
):
    """
    Download torrents and perform cleanup based on conditions.

    Args:
    - ses (lt.session): libtorrent session
    - downloads (list): List of torrent objects
    - destination (str): Destination folder for downloaded files
    - magnet_links (set): Set of magnet links
    - magnet_links_txt (str): Path to the text file containing magnet links
    - logger (logging.Logger): Logger object
    """

    start_time = time.time()  # Record the start time
    logging.info(f"Starting download with {len(downloads)} torrent(s)")

    # Loop until all torrents are processed
    while downloads:
        for torrent in downloads.copy():
            # Check if torrent is stale and remove if necessary
            if torrent.remove_stale_torrents(ses):
                torrent.remove_files()
                torrent_name = torrent.name
                timeout = torrent.timeout
                downloads.remove(torrent)
                logger.info(
                    f"Removed torrent from session due to {timeout//3600} hours runtime: {torrent_name}"
                )
                continue

            # Check if torrent is still downloading
            if not torrent.get_status().is_seeding:
                # Display the torrent progress and download speed
                display_torrent_progress(torrent)

            # Torrent is successfully downloaded
            else:
                if torrent.move_video_files(destination):
                    torrent.remove_torrent(ses)
                    torrent.remove_the_link_from_the_list(magnet_links)
                    update_txt_file(magnet_links_txt, magnet_links, logger)
                    torrent_name = torrent.name
                    downloads.remove(torrent)
                    logger.info(f"Successfully downloaded: {torrent_name}")

                else:
                    torrent.remove_torrent(ses)
                    downloads.remove(torrent)

                gc.collect()

        # Calculate elapsed time
        elapsed_time = time.time() - start_time
        hours, remainder = divmod(elapsed_time, 3600)
        minutes, _ = divmod(remainder, 60)

        # Display the number of torrents still downloading and elapsed time
        pprint(
            f"{time.strftime('%H:%M:%S')} - Torrents downloading: {len(downloads)}. Elapsed time: {int(hours)} hours, {int(minutes)} minutes."
        )

    logger.info("All downloads processed.")

In [ ]:
def setup_logging(destination_path: str = "/content/drive/MyDrive/"):
    """
    Set up logging with coloredlogs for console output and standard logging to a file,
    ensuring that log messages are not duplicated.

    Args:
    - destination_path (str): The path where the log file will be saved.
    """

    # create a folder for logs if it doesn't exist
    destination_path = os.path.join(destination_path, "logs")

    if not os.path.exists(destination_path):
        os.makedirs(destination_path)

    current_time = time.strftime("%Y-%m-%d_%H-%M-%S")
    log_filename = f"log_{current_time}.log"
    log_file_path = os.path.join(destination_path, log_filename)

    logger = logging.getLogger(__name__)
    logger.setLevel(logging.INFO)

    # Check if logger already has handlers and remove them to prevent duplicate logs
    if logger.hasHandlers():
        logger.handlers.clear()

    file_handler = logging.FileHandler(log_file_path)
    file_handler.setLevel(logging.INFO)
    formatter = logging.Formatter(
        "%(asctime)s - %(levelname)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S"
    )
    file_handler.setFormatter(formatter)
    logger.addHandler(file_handler)

    coloredlogs.install(
        level="INFO",
        logger=logger,
        fmt="%(asctime)s - %(levelname)s - %(message)s",
        datefmt="%Y-%m-%d %H:%M:%S",
    )

    logger.info("Logging setup complete. Log file created at: {}".format(log_file_path))

    return logger

In [ ]:
def save_state(
    downloads: list,
    state_file_path: str = "/content/session_state.json",
):
    with open(state_file_path, "w") as f:
        # Assuming downloads list can be serialized directly; may need customization
        json.dump(downloads, f)

In [ ]:
def load_state(
    state_file_path: str = "/content/session_state.json",
):
    """
    Loads the application state from a JSON file.

    This function checks if the specified state file exists and is not empty. If the file exists and contains valid JSON,
    the function returns the decoded JSON object. If the file is empty or does not exist, it returns an empty list.
    If there is an error decoding the JSON (indicating corruption or invalid format), it also returns an empty list and
    prints an error message.

    Args:
    - state_file_path (str): The path to the state file. Defaults to a specific path in Google Drive.

    Returns:
    - list/dict: The loaded state from the file if successful, or an empty list if the file is empty, does not exist, or
                 contains invalid JSON.
    """

    # Check if the file exists at the specified path
    if os.path.exists(state_file_path):
        # Check if the file is empty by comparing its size to 0
        if os.path.getsize(state_file_path) > 0:
            try:
                # Attempt to open and load the JSON content from the file
                with open(state_file_path, "r") as f:
                    return json.load(f)
            except json.JSONDecodeError:
                # Catch decoding errors, which indicate the file is not valid JSON
                logging.error(
                    f"Error decoding JSON from file {state_file_path}. File might be corrupted."
                )
                return []  # Return an empty list in case of JSON decoding errors
        else:
            # File exists but is empty, so log a message and return an empty list
            logging.error(f"File {state_file_path} exists but is empty.")
            return []
    else:
        # File does not exist at the path, so log a message and return an empty list
        logging.error(f"File {state_file_path} does not exist.")
        return []

In [ ]:
def main(
    magnet_links_txt: str,
    ses: lt.session,
    destination: str,
):
    logger = setup_logging(destination)

    try:
        start_time = time.time()

        logger.info("Starting the torrent download process...")

        # load_state
        downloads = load_state()

        # Load magnet links from txt file
        magnet_links = load_magnet_links(magnet_links_txt, logger)
        if not magnet_links:
            logger.warning("No magnet links loaded. Exiting...")
            return

        # Get additional magnet links from user input (consider automating or optimizing this step based on your context)
        # magnet_links = get_magnet_links(magnet_links) # Commented out for optimization

        # Update the magnet links file
        update_txt_file(magnet_links_txt, magnet_links, logger)

        # Create torrents and start downloading
        create_torrents(ses, magnet_links, downloads, logger)
        check_already_downladed(
            ses, downloads, destination, magnet_links, magnet_links_txt, logger
        )

        # save created torrent states
        # save_state(downloads)

        download_torrents(
            ses, downloads, destination, magnet_links, magnet_links_txt, logger
        )

        # Summarize the operation
        elapsed_time_minutes = (time.time() - start_time) // 60
        logger.info(f"Torrents download completed in {elapsed_time_minutes} minutes.")

        runtime.unassign()

    except Exception:
        logger.exception("An error occurred in main thread:")

In [ ]:
if __name__ == "__main__":
    ses = lt.session(
        {
            "user_agent": f"python_client/{lt.__version__}",
            "listen_interfaces": "0.0.0.0:6881",
        }
    )

    magnet_links_txt = r"/content/drive/MyDrive/Torrents/magnet_links.txt"
    destination = r"/content/drive/MyDrive/Torrents"

    main(magnet_links_txt, ses, destination)